In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
df_sample = df_scaled.sample(n=10000, replace=False)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

In [33]:
psb = PairsBootstrap(Y_data, X_data, iter = 1000)
psb.fit()

In [34]:
rsb = ResidualBootstrap(Y_data, X_data, iter = 1000)
rsb.fit()

In [35]:
wb_stdn = WildBootstrap(Y_data, X_data, iter = 1000, from_distro = 'standard_normal')
wb_stdn.fit()

In [36]:
wb_rad = WildBootstrap(Y_data, X_data, iter = 1000, from_distro = 'rademacher')
wb_rad.fit()

In [37]:
wb_mam = WildBootstrap(Y_data, X_data, iter = 1000, from_distro = 'mammen')
wb_mam.fit()

In [38]:
psb.summary()

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                      Pairs Bootstrap results with sample size of 10000 and bootstrap resampling size of 1000 using 95.00% CI                                       |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |       PCT CI       |  PCT CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     const      |     6.2034    |             6.2042            |  0.0009  |      0.

In [39]:
rsb.summary()

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                     Residual Bootstrap results with sample size of 10000 and bootstrap resampling size of 1000 using 95.00% CI                                     |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |       PCT CI       |  PCT CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     const      |     6.2034    |             6.2015            |  0.0019  |      0.

In [40]:
wb_stdn.summary()

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                            Wild Bootstrap with Standard Normal results with sample size of 10000 and bootstrap resampling size of 1000 using 95.00% CI                             |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |       PCT CI       |  PCT CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     const      |     6.2034    |             6.2034            |  0.0001  |      0.

In [41]:
wb_rad.summary()

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                               Wild Bootstrap with Rademacher results with sample size of 10000 and bootstrap resampling size of 1000 using 95.00% CI                               |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |       PCT CI       |  PCT CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     const      |     6.2034    |             6.2023            |  0.0011  |      0.

In [42]:
wb_mam.summary()

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                 Wild Bootstrap with Mammen results with sample size of 10000 and bootstrap resampling size of 1000 using 95.00% CI                                 |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     Params     |  Orig Coeffs  |  Mean of Bootstrapped Coeffs  |   Bias   |  Orig Coeff SE  |  SE of Bootstrapped Coeffs  |  % of Diff in SE  |       PCT CI       |  PCT CI Diff  |
+----------------+---------------+-------------------------------+----------+-----------------+-----------------------------+-------------------+--------------------+---------------+
|     const      |     6.2034    |             6.2042            |  0.0009  |      0.